# MODNet on experimental dataset 

MODNet model with the GA, and doing joint learning on the experimental and PBE band gaps.

In [1]:
def setup_threading():
    import os
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["TF_NUM_INTRAOP_THREADS"] = "1"
    os.environ["TF_NUM_INTEROP_THREADS"] = "1"
    os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
setup_threading()

In [3]:
from modnet.preprocessing import MODData
from modnet.models.vanilla import MODNetModel
from modnet.hyper_opt.fit_genetic import FitGenetic
import numpy as np
import os
import copy

from collections import defaultdict
import itertools
import os
import pandas as pd
import matplotlib.pyplot as plt 
from IPython.display import Markdown
from matminer.datasets import load_dataset, get_all_dataset_info
from pymatgen.core import Composition

from modnet.featurizers import MODFeaturizer
from modnet.featurizers.presets import DeBreuck2020Featurizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


If you use the ChemEnv tool for your research, please consider citing the following reference(s) :
David Waroquiers, Xavier Gonze, Gian-Marco Rignanese, Cathrin Welker-Nieuwoudt, Frank Rosowski,
Michael Goebel, Stephan Schenk, Peter Degelmann, Rute Andre, Robert Glaum, and Geoffroy Hautier,
"Statistical analysis of coordination environments in oxides",
Chem. Mater., 2017, 29 (19), pp 8346-8360,
DOI: 10.1021/acs.chemmater.7b02766



In [4]:
from sklearn.model_selection import KFold
from modnet.preprocessing import MODData

def shuffle_MD(data,random_state=10):
    data = copy.deepcopy(data)
    ids = data.df_targets.sample(frac=1,random_state=random_state).index
    data.df_featurized = data.df_featurized.loc[ids]
    data.df_targets = data.df_targets.loc[ids]
    data.df_structure = data.df_structure.loc[ids]
    
    return data

def MDKsplit(data,n_splits=5,random_state=10):
    data = shuffle_MD(data,random_state=random_state)
    ids = np.array(data.structure_ids)
    kf = KFold(n_splits=n_splits,shuffle=True,random_state=random_state)
    folds = []
    for train_idx, val_idx in kf.split(ids):
        data_train = MODData(data.df_structure.iloc[train_idx]['structure'].values,data.df_targets.iloc[train_idx].values,target_names=data.df_targets.columns,structure_ids=ids[train_idx])
        data_train.df_featurized = data.df_featurized.iloc[train_idx]
        #data_train.optimal_features = data.optimal_features
        
        data_val = MODData(data.df_structure.iloc[val_idx]['structure'].values,data.df_targets.iloc[val_idx].values,target_names=data.df_targets.columns,structure_ids=ids[val_idx])
        data_val.df_featurized = data.df_featurized.iloc[val_idx]
        #data_val.optimal_features = data.optimal_features

        folds.append((data_train,data_val))
        
    return folds

In [5]:
md = MODData.load('exp_gap_all')
df = pd.read_pickle('df_exp_cleaned.pkl')
md_joint = MODData.load('exp_pbe_joint')

2021-06-14 17:43:36,444 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fdd646de6d0> object, created with modnet version <=0.1.7
2021-06-14 17:43:37,049 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fddd4356370> object, created with modnet version <=0.1.7


In [6]:
md_joint.df_targets

,exp_gap,pbe_gap,difference
mp-12699,0.00,0.0000,0.0000
mp-559459,3.40,2.1704,1.2296
mp-21162,0.00,0.0000,0.0000
mp-1306,0.00,0.0000,0.0000
mp-15252,0.53,0.1049,0.4251
...,...,...,...
mp-556541,1.80,1.5135,0.2865
mp-2602,1.04,0.0000,1.0400
mp-30366,0.00,0.0000,0.0000
mp-3718,1.45,0.2418,1.2082


In [7]:
md_joint.df_targets = md_joint.df_targets.drop(columns='difference')

In [8]:
md_joint.df_targets

,exp_gap,pbe_gap
mp-12699,0.00,0.0000
mp-559459,3.40,2.1704
mp-21162,0.00,0.0000
mp-1306,0.00,0.0000
mp-15252,0.53,0.1049
...,...,...
mp-556541,1.80,1.5135
mp-2602,1.04,0.0000
mp-30366,0.00,0.0000
mp-3718,1.45,0.2418


In [9]:
md_joint.df_structure

,structure
id,
mp-12699,"[[ 0.13616363 -2.60631371 1.50340165] Ca, [-1..."
mp-559459,"[[2.14049282 6.43464803 1.76199846] Cs, [-0.70..."
mp-21162,"[[0. 0. 0.] Sr, [0. 2.5259875 2.5259875..."
mp-1306,"[[3.259129 1.88166243 1.15971025] Th, [ 3.25..."
mp-15252,"[[0.95531657 1.042239 0.86229681] Cu, [2.803..."
...,...
mp-556541,"[[0. 6.49363948 5.82657217] In, [ 0. ..."
mp-2602,"[[3.023318 3.023318 3.023318] Nd, [0. 0. 0.] As]"
mp-30366,"[[4.51774558 1.37529211 4.32659015] Ca, [1.109..."


In [10]:
import uncertainty_toolbox as uct

k = 5
random_state = 202010
folds = MDKsplit(md_joint,n_splits=k,random_state=random_state)
maes = np.ones(5)
uncertainties = np.ones(5)
metricss = []
for i,f in enumerate(folds):
    train = f[0]
    test = f[1]
    #train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
    fpath = 'train_{}_{}'.format(random_state,i+1)
    if os.path.exists(fpath):
        train = MODData.load(fpath)
    else:
        train.feature_selection(n=-1, use_precomputed_cross_nmi=True)
        train.save(fpath)
      
    # assure no overlap
    assert len(set(train.df_targets.index).intersection(set(test.df_targets.index))) == 0
    
    ga = FitGenetic(train)
    model = ga.run(size_pop=20, num_generations=10, n_jobs=10)
    
    pred, std = model.predict(test, return_unc=True)
    true = test.df_targets
    metrics = uct.metrics.get_all_metrics(pred['exp_gap'].values, std['exp_gap'].values, true['exp_gap'].values)
    error = pred['exp_gap']-true['exp_gap']
    error = error.drop(pred.index[((pred['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    std = std['exp_gap'].drop(std.index[((std['exp_gap']).abs()>20)]) # drop unrealistic values: happens extremely rarely
    mae = np.abs(error.values).mean()
    uncertainty = np.abs(std).mean()
    print('MAE =')
    print(mae)
    print('uncertainty =')
    print(uncertainty)
    maes[i] = mae
    uncertainties[i] = uncertainty
    metricss.append(metrics)

2021-06-14 17:43:37,219 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,234 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,243 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,258 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,267 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,281 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,290 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,305 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,314 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:37,328 - modnet - INFO - Loaded DeBreuck2020Featurizer featurizer.
2021-06-14 17:43:38,112 - modnet - INFO - Loaded <modnet.preprocessing.MODData object at 0x7fdd5225e430> object, created with modnet version 0.1.9
2021-06-14 17

100%|██████████| 100/100 [11:58<00:00,  7.18s/it]

2021-06-14 17:55:37,819 - modnet - INFO - Loss per individual: ind 0: 0.951 	ind 1: 0.634 	ind 2: 0.634 	ind 3: 0.652 	ind 4: 0.619 	ind 5: 0.753 	ind 6: 0.613 	ind 7: 0.638 	ind 8: 0.704 	ind 9: 0.585 	ind 10: 0.646 	ind 11: 0.951 	ind 12: 0.736 	ind 13: 0.633 	ind 14: 0.625 	ind 15: 0.745 	ind 16: 0.612 	ind 17: 0.711 	ind 18: 0.775 	ind 19: 0.693 	


2021-06-14 17:55:39,330 - modnet - INFO - Generation number 1
2021-06-14 17:55:41,619 - modnet - INFO - Multiprocessing on 10 cores. Total of 128 cores available.


100%|██████████| 100/100 [14:48<00:00,  8.88s/it]

...

 (3/n) Calculating adversarial group calibration metrics
  [1/2] for mean absolute calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


 10%|█         | 1/10 [00:00<00:01,  6.29it/s]

  [2/2] for root mean squared calibration error
Measuring adversarial group calibration by spanning group size between 0.0 and 1.0, in 10 intervals


100%|██████████| 10/10 [00:01<00:00,  6.36it/s]


 (4/n) Calculating sharpness metrics
 (n/n) Calculating proper scoring rule metrics
**Finished Calculating All Metrics**


===================== Accuracy Metrics =====================
  MAE           0.365
  RMSE          0.725
  MDAE          0.039
  MARPD         137.524
  R2            0.827
  Correlation   0.910
=============== Average Calibration Metrics ================
  Root-mean-squared Calibration Error   0.131
  Mean-absolute Calibration Error       0.114
  Miscalibration Area                   0.115
========== Adversarial Group Calibration Metrics ===========
  Mean-absolute Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.163
     Group Size: 0.56 -- Calibration Error: 0.127
     Group Size: 1.00 -- Calibration Error: 0.114
  Root-mean-squared Adversarial Group Calibration Error
     Group Size: 0.11 -- Calibration Error: 0.185
     Group Size: 0.56 -- Calibration Error: 0.142
     Group Size: 1.00 -- Calibration Error: 0.131
==============

In [11]:
maes

array([0.36664176, 0.36490418, 0.2906571 , 0.33308754, 0.36468457])

In [12]:
maes.mean()

0.3439950288211245

In [13]:
uncertainties

array([0.19157691, 0.23580144, 0.18824559, 0.18684244, 0.16942088])

In [14]:
uncertainties.mean()

0.1943774551153183

In [15]:
metricss

[{'accuracy': {'mae': 0.3666417588051334,
   'rmse': 0.7640836480782559,
   'mdae': 0.06406853348016739,
   'marpd': 130.92784034187395,
   'r2': 0.7738235314172658,
   'corr': 0.885975733070209},
  'avg_calibration': {'rms_cal': 0.07719689281185402,
   'ma_cal': 0.048403388725969394,
   'miscal_area': 0.04885096242067718},
  'adv_group_calibration': {'ma_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ]),
    'adv_group_cali_mean': array([0.40063131, 0.11951781, 0.10728575, 0.08403695, 0.07983367,
           0.07316017, 0.06545201, 0.06012563, 0.0570129 , 0.04840339]),
    'adv_group_cali_stderr': array([0.06944661, 0.02622863, 0.02214075, 0.01212431, 0.01095031,
           0.00803942, 0.00786954, 0.00352915, 0.00370456, 0.        ])},
   'rms_adv_group_cal': {'group_sizes': array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
           0.55555556, 0.6

#### Conclusion

Relative percentage change of the error compared to MODNet_exp_GA reference: 2%